# YoloV5 Test
## Generate Image Dataset
### Imports

In [5]:
import cv2
import torch
import os
import numpy as np
from modules import image_processing as ip

### Variable Settings

In [6]:
img_size= 240

img_folder_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
image_name = '1.jpg'
image_path = img_folder_path + '\\' + image_name

model_weights_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\yolov5_temp\\runs\\train\\Big\\weights\\best.pt'

labels = ['OverlapingObjects', 'Nut', 'Screw', 'Washer']

### Load YoloV5-Model pretrained with own data

In [7]:
def load_model(path):
    # Lade das trainierte Modell
    print(path)
    #model = torch.load(model_weights_path, map_location='cpu')
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=path)
    return model


model = load_model(path=model_weights_path)

C:\Users\doett\Documents\Workspace\GitHub\proj-sorting-robot\Testing\yolov5_temp\runs\train\Big\weights\best.pt


Using cache found in C:\Users\doett/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-12-17 Python-3.11.5 torch-2.1.2+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


### Load Images

In [125]:
image_path =  'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps\\1.jpg'

image = cv2.imread(image_path)

# Überprüfen, ob das Bild erfolgreich geladen wurde
if image is not None:
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("[WARNING] No image loaded")
    

### Load more files of one Folder

In [123]:
img_folder_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
img_array = []
# Überprüfen, ob der Ordner existiert
if os.path.isdir(img_folder_path):
    # Durchlaufe alle Dateien im Ordner
    for filename in os.listdir(img_folder_path):
        # Pfade für jedes Object erstellen
        image_path = os.path.join(img_folder_path, filename)

        # Überprüfen, ob es sich um eine Datei handelt
        if os.path.isfile(image_path):
            # Load Image
            image = cv2.imread(image_path)

            # Überprüfen, ob das Bild erfolgreich geladen wurde
            if image is not None:
                img_array.append((image, filename))
                cv2.imshow('Image', image)
            else:
                print(f"Unable to load {filename}")
        else:
            print(f"No File: {filename} ")
else:
    print("Path don't exists")

cv2.waitKey(0)
cv2.destroyAllWindows()

Unable to load 1.json
Unable to load 10.json
Unable to load 11.json
Unable to load 12.json
Unable to load 13.json
Unable to load 14.json
Unable to load 15.json
Unable to load 16.json
Unable to load 17.json
Unable to load 18.json
Unable to load 19.json
Unable to load 2.json
Unable to load 20.json
Unable to load 21.json
Unable to load 22.json
Unable to load 23.json
Unable to load 24.json
Unable to load 25.json
Unable to load 26.json
Unable to load 27.json
Unable to load 28.json
Unable to load 29.json
Unable to load 3.json
Unable to load 30.json
Unable to load 31.json
Unable to load 32.json
Unable to load 33.json
Unable to load 34.json
Unable to load 35.json
Unable to load 36.json
Unable to load 37.json
Unable to load 38.json
Unable to load 39.json
Unable to load 4.json
Unable to load 40.json
Unable to load 41.json
Unable to load 42.json
Unable to load 43.json
Unable to load 44.json
Unable to load 45.json
Unable to load 46.json
Unable to load 47.json
Unable to load 48.json
Unable to load 

In [10]:
imageNr = 0

def get_image(imagenr):
    return img_array[imagenr][0]

def get_image_name(imagenr):
    return img_array[imagenr][1]

print(get_image_name(imageNr))
print(get_image(imageNr))

cv2.imshow('{}'.format(get_image_name(imageNr)), get_image(imageNr))
cv2.waitKey(0)
cv2.destroyAllWindows()


1.jpg
[[[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 [[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 [[18 20  8]
  [19 21  9]
  [22 22 10]
  ...
  [58 37 10]
  [52 26  9]
  [45 19  3]]

 ...

 [[15  8 11]
  [16  9 12]
  [17 10 13]
  ...
  [ 5 13 12]
  [ 5 13 12]
  [ 3 14 12]]

 [[13  6 11]
  [14  7 12]
  [16  9 14]
  ...
  [ 6 14 13]
  [ 3 14 12]
  [ 1 14 12]]

 [[14  7 12]
  [15  8 13]
  [17 10 15]
  ...
  [ 5 16 14]
  [ 4 15 13]
  [ 1 14 12]]]


### Predict Results

In [151]:
image = cv2.imread(image_path)
image = image[175:950, 0:1700]
output = image.copy()
results = model(image)

cropped_boxes = []  # Liste für ausgeschnittene Boxen
con_array = []

for box in results.xyxy[0].cpu().numpy():
    box = [float(i) for i in box]
    xmin, ymin, xmax, ymax, conf, cls = box
    
    region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
    cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(cvt, 13)
    
    _, otsu_thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sortiere Konturen nach Fläche
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Nutze nur die größte Kontur
    if contours:
        contour = contours[0]
        con_array.append(contour)
        
        # Berechne das rotierte Rechteck
        rect = cv2.minAreaRect(contour)
        box_points = cv2.boxPoints(rect)
        box_points = np.int0(box_points)
        
        # Offset basierend auf den Box-Koordinaten
        box_points[:, 0] += int(xmin)
        box_points[:, 1] += int(ymin)
        cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
        
    output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                         (0, 255, 0), 1, cv2.LINE_AA)    
    
rectangles = ip.get_rects_from_contours(con_array)
bounding_boxes = ip.get_bounding_boxes_from_rectangles(rectangles)
object_images = ip.warp_objects_horizontal(image, rectangles, bounding_boxes)
cv2.imshow('test',object_images[1])

cropped_boxes.append( ip.standardize_images(object_images))
black_BG = cropped_boxes

for i, image in enumerate(black_BG):
    path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\Scanned Data\\Part{}.png'.format(i)
    cv2.imwrite(path, image)
    
cv2.imshow("Predicted Objects", output)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\doett\AppData\Local\Temp\ipykernel_20176\4140732433.py:32: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)


In [126]:
image = cv2.imread(image_path)
image = image[175:950, 0:1700]
output = image.copy()
results = model(image)

cropped_boxes = []  # Liste für ausgeschnittene Boxen

for box in results.xyxy[0].cpu().numpy():
    box = [float(i) for i in box]
    xmin, ymin, xmax, ymax, conf, cls = box
    
    region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
    cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(cvt, 13)
    
    _, otsu_thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    con_array = []
    
    # Sortiere Konturen nach Fläche
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Nutze nur die größte Kontur
    if contours:
        contour = contours[0]
        
        # Berechne das rotierte Rechteck
        rect = cv2.minAreaRect(contour)
        box_points = cv2.boxPoints(rect)
        box_points = np.int0(box_points)
        
        # Offset basierend auf den Box-Koordinaten
        box_points[:, 0] += int(xmin)
        box_points[:, 1] += int(ymin)
        cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
        
        # Berechne die Rotationsmatrix
        center = (xmin + (xmax - xmin) / 2, ymin + (ymax - ymin) / 2)
        M = cv2.getRotationMatrix2D(center, rect[2], 1.0)
        
        # Rotiere das Bild
        rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        
        # Berechne die Bounding Box um die Rotated Bounding Box
        x, y, w, h = cv2.boundingRect(box_points)
        
        # Ausschneiden und Speichern des Bereichs
        cropped_box = rotated[y:y+h, x:x+w]
        
        if h > w:
            cropped_box = cv2.rotate(cropped_box, cv2.ROTATE_90_CLOCKWISE)
            cropped_boxes.append(cropped_box)
            
        else:
            cropped_boxes.append(cropped_box)
        
    output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                         (0, 255, 0), 1, cv2.LINE_AA)    

black_BG = ip.standardize_images(cropped_boxes)

for i, image in enumerate(black_BG):
    path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\Scanned Data\\Part{}.png'.format(i)
    cv2.imwrite(path, image)
    
cv2.imshow("Predicted Objects", output)
cv2.waitKey(0)
cv2.destroyAllWindows()



C:\Users\doett\AppData\Local\Temp\ipykernel_20176\2006893766.py:31: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box_points = np.int0(box_points)


In [117]:
image = cv2.imread(image_path)
output = image.copy()
results = model(image)


# Extract Conventional

In [2]:
DataFolderPath = 'Testing/Scanned Data'

In [3]:
image = cv2.imread(image_path)
output = image.copy()
results = model(image)

NameError: name 'image_path' is not defined

In [4]:
preprocessed_image = ip.image_preprocessing(image)
cv2.imshow('Test',preprocessed_image)
contours, rectangles, bounding_boxes, object_images = ip.get_objects_in_preprocessed_image(preprocessed_image,
                                                                                             smaller_image_area=True)
black_BG = ip.standardize_images(object_images,224)

#cv2.imshow('test',black_BG[0])

canvas_image = cv2.drawContours(preprocessed_image, bounding_boxes, -1, (0, 0, 255), 2)
cv2.imshow('canvas_image',canvas_image)

cv2.waitKey(0)
cv2.destroyAllWindows() 


    